In [ ]:
# --------------------------------- Technical indicators
def cci(pair,cci_sma,cci_num): # CCI indicator
    cci_num=0.015
    pair['typ.price']=(pair['high']+pair['low']+pair['close'])/3
    pair['cci']=(pair['typ.price']-pair['typ.price'].rolling(cci_sma).mean())/(cci_num*stdev(pair['typ.price']))

def atr(pair,period): # ATR indicator
    pair['atr']=ta.atr(pair['high'],pair['low'],pair['close'],period)

def willr(pair,period): # Williams R indicator
    pair['willr']=ta.willr(pair['high'],pair['low'],pair['close'],period)
    
def adx(pair,period): # ADX indicator
    pair[['ADX_14','DMP_14','DMN_14']]=ta.adx(pair['high'],pair['low'],pair['close'],period)

def macd(pair,slow,fast,smooth): # MACD indicator
    pair['macd_slow']=pair['close'].ewm(span=fast).mean()-pair['close'].ewm(span=slow).mean()
    pair['macd_fast']=pair['close'].ewm(span=smooth).mean()
    
def Stochastic(pair,k_period,d_period): # Stochastic Oscillator indicator
    pair['sto_dif1']=pair['close']-pair['low'].rolling(k_period).min()
    pair['sto_dif2']=pair['high'].rolling(k_period).max()-pair['low'].rolling(k_period).min()
    pair['sto_k']=pair['sto_dif1']/pair['sto_dif2']*100
    pair['sto_main']=pair['sto_dif1'].rolling(d_period).sum()/pair['sto_dif2'].rolling(d_period).sum()*100
    pair['sto_signal']=pair['sto_main'].rolling(d_period).mean()

def MA(pair,period,type_ma): # Moving Average indicator
    if type_ma=="sma":
        pair['sma'+str(period)]=pair['close'].rolling(period).mean()
    elif type_ma=="ema":
        pair['ema'+str(period)]=pair['close'].ewm(span=period).mean()
        
def RSI(pair, periods): # RSI indicator
    pair['rsi_up']=pair['close']-pair['close'].shift(1) #shift(1)-previous, shift(-1)-next
    pair['rsi_down']=pair['close'].shift(1)-pair['close']
    pair.loc[pair['rsi_up'] <= 0, 'rsi_up'] = 0
    pair.loc[pair['rsi_down'] <= 0, 'rsi_down'] = 0
    pair['rsi_up_avg']=pair['rsi_up'].rolling(periods).mean()#.apply(lambda x: x[x!= 0].mean())
    pair['rsi_down_avg']=pair['rsi_down'].rolling(periods).mean()#.apply(lambda x: x[x!= 0].mean())
    for i in range(periods+1,len(pair)):
        pair['rsi_up_avg'][i]=(pair['rsi_up_avg'][i-1]*(periods-1)+pair['rsi_up'][i])/periods
        pair['rsi_down_avg'][i]=(pair['rsi_down_avg'][i-1]*(periods-1)+pair['rsi_down'][i])/periods
    pair['rsi_rs']=pair['rsi_up_avg']/pair["rsi_down_avg"]
    pair['rsi']=100-(100/(1+pair['rsi_rs']))
    
def BB(pair,avgday,stdmul): # Bollinger Band indicator
    pair['bb_std']=pair['close'].rolling(avgday).std()
    pair['bb_mid']=pair['close'].rolling(avgday).mean()
    pair['bb_down']=pair['bb_mid']-pair['bb_std']*stdmul
    pair['bb_up']=pair['bb_mid']+pair['bb_std']*stdmul

def ASD(return_in): # Annual standart deviation
  var_output=return_in.var()
  var_output=var_output*252**(1/2) # 4hour - 1512
  return var_output

def calculate_max_drawdown(returns): # Max Drawdown calculation
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns / rolling_max) - 1
    max_drawdown = drawdown.min()
    return max_drawdown

def calculate_sharpe_ratio(return_series,rf): # Sharpe ratio calculation
    x=return_series-rf
    mean=x.mean()
    sigma = return_series.std()
    return mean / sigma

def calculate_roi(equity): # Return on Investment calculation
  x=(equity.iloc[-1]-equity.iloc[0])/equity.iloc[0]
  return x

for a in [EURUSD, AUDUSD, GBPUSD, NZDUSD,USDCAD,USDCHF]: # Applying technical indicators
    MA(a,10,'ema')
    MA(a,20,'ema')
    MA(a,50,"ema")
    MA(a,200,"ema")
    MA(a,100,"ema")
    macd(a,26,12,9)
    RSI(a, 14)
    Stochastic(a,14,3)
    cci(a,20,0.015)
    BB(a,20,2)
    atr(a,14)
    willr(a,14)
    adx(a,14)
    a=a.drop(columns=['sto_dif1', 'sto_dif2','tick_volume', 'spread',
                      'real_volume','rsi_up', 'rsi_down', 'rsi_up_avg', 'rsi_down_avg',
                      'rsi_rs','sto_k','bb_std','typ.price'], axis=1, inplace=True)